## Introduction

Geometry is not plasma physics, but it isn't trivial either. Chances are most of
your day-to-day interaction with bluemira will revolve around geometry in some form
or another. Puns intended.

There a few basic concepts you need to familiarise yourself with:
* Basic objects: [`BluemiraWire`, `BluemiraFace`, `BluemiraShell`, `BluemiraSolid`]
* Basic properties
* Matryoshka structure
* Geometry creation
* Geometry modification
* Geometry operations

## Imports

Let's start out by importing all the basic objects, and some typical tools

In [ ]:
import numpy as np

# Some useful tools
import bluemira.geometry.tools as geotools

# Some display functionality
from bluemira.display import plot_2d, show_cad
from bluemira.display.displayer import DisplayCADOptions
from bluemira.geometry.face import BluemiraFace
from bluemira.geometry.shell import BluemiraShell
from bluemira.geometry.solid import BluemiraSolid

# Basic objects
from bluemira.geometry.wire import BluemiraWire


## Make a cylinder

There are many ways to make a cylinder, but perhaps the simplest way is as follows:
* Make a circular Wire
* Make a Face from that Wire
* Extrude that Face along a vector, to make a Solid

In [ ]:
# Note that we are going to give these geometries some labels, which
# we might use later.
circle_wire = geotools.make_circle(
    radius=5,
    center=(0, 0, 0),
    axis=(0, 0, 1),
    start_angle=0,
    end_angle=360,
    label="my_wire",
)
circle_face = geotools.make_face(circle_wire, label="my_face")
cylinder = geotools.extrude_shape(circle_face, vec=(0, 0, 10), label="my_solid")


## Simple properties and representations

In [ ]:
# Let's start off with some simple properties
print(f"Circle length: {circle_wire.length} m")
print(f"Circle area: {circle_face.area} m^2")
print(f"Cylinder volume: {cylinder.volume} m^3")

# You can also just print or repr these objects to get some useful info
print(cylinder)

## Display

Geometry objects can be displayed via `show_cad`, and the appearance
of said objects customised by specifying `color` and `transparency`.

In [ ]:
show_cad(cylinder, DisplayCADOptions(color="blue", transparency=0.1))

## BluemiraGeo structure

Each BluemiraGeo object has access to its sub-components that form the geoemtry

i.e: Solid -> Shell -> Face -> Wire

Let's take a little peek under the hood of our cylinder

In [ ]:
print(f"Our cylinder is a BluemiraSolid: {isinstance(cylinder, BluemiraSolid)}")

print(f"Cylinder shell: {cylinder.shells}")
print(f"Cylinder faces: {cylinder.faces}")
print(f"Cylinder wires: {cylinder.wires}")
print(f"Cylinder vertixes: {cylinder.vertexes}")

OK, so a cylinder is pretty simple, but more complicated shapes
will follow the same pattern.

It does go deeper than this, but that is outside the intended
user-realm.

## Geometry creation

Let's get familiar with some more ways of making geometries. We've
looked at circle already, but what else is out there:
* polygons
* splines
* a bit of everything

In [ ]:
# Polygon
theta = np.linspace(0, 2 * np.pi, 6)
x = 5 * np.cos(theta)
y = np.zeros(6)
z = 5 * np.sin(theta)

points = np.array([x, y, z])
pentagon = geotools.make_polygon(points)

plot_2d(pentagon)

Polygons are good for things with straight lines.
Circles you've met already.
For everything else, there's splines.

Say you have a weird shape, that you might calculate via a equation.
It's not a good idea to make a polygon with lots of very small sides
for this. It's computationally expensive, and it will look ugly.

In [ ]:
# Spline

x = np.linspace(0, 10, 1000)
y = 0.5 * np.sin(x) + 3 * np.cos(x) ** 2
z = np.zeros(1000)

points = np.array([x, y, z])
spline = geotools.interpolate_bspline(points)
points = np.array([x, y + 3, z])
polygon = geotools.make_polygon(points)

show_cad(
    [spline, polygon], [DisplayCADOptions(color="blue"), DisplayCADOptions(color="red")]
)

In [ ]:
# To get an idea of why polygons are bad / slow / ugly, try:
vector = (0, 0, 1)
show_cad(
    [geotools.extrude_shape(spline, vector), geotools.extrude_shape(polygon, vector)],
    [DisplayCADOptions(color="blue"), DisplayCADOptions(color="red")],
)

In [ ]:
# There is nothing stopping you from combining different primitives, though!

radius = 2
part_circle = geotools.make_circle(radius=radius, start_angle=0, end_angle=270)

points = np.array([[radius, 0, 0], [0, 0, -radius], [0, 0, 0]])
closure = geotools.make_polygon(points)

my_shape = BluemiraWire(part_circle + closure)

# Let's just check we got that right...
print(f"My shape is closed: {my_shape.is_closed()}")

show_cad(geotools.make_face(my_shape))

## Geometry operations: Part 1
Making 3-D shapes from 2-D shapes

You can:
* extrude a shape `extrude_shape`, as we did with our cylinder
* revolve a shape `revolve_shape`
* sweep a shape `sweep_shape`

In [ ]:
# Make a hollow cylinder, by revolving a rectangle
points = np.array([[4, 5, 5, 4], [0, 0, 0, 0], [2, 2, 3, 3]])
rectangle = geotools.make_face(geotools.make_polygon(points, closed=True))

hollow_cylinder = geotools.revolve_shape(
    rectangle, base=(0, 0, 0), direction=(0, 0, 1), degree=360
)

show_cad(hollow_cylinder)

In [ ]:
# Sweep a profile along a path

points = np.array([[4.5, 4.5], [0, 3], [2.5, 2.5]])
straight_line = geotools.make_polygon(points)
quarter_turn = geotools.make_circle(
    center=(3, 3, 2.5), axis=(0, 0, 1), radius=1.5, end_angle=90
)
path = BluemiraWire(straight_line + quarter_turn)
solid = geotools.sweep_shape(rectangle.wires, path)
show_cad(solid)

## Geometry operations: Part 2
Making 3-D shapes from 3-D shapes

Boolean operations often come in very useful when making CAD.
* You can join geometries together with `boolean_fuse`
* You can cut geometries from one another with `boolean_cut`

In [ ]:
points = np.array(
    [
        [0, 2, 2, 0],
        [0, 0, 0, 0],
        [0, 0, 3, 3],
    ]
)

box_1 = geotools.make_face(geotools.make_polygon(points, closed=True))
box_1 = geotools.extrude_shape(box_1, (0, 2, 0))

points = np.array(
    [
        [1, 3, 3, 1],
        [0, 0, 0, 0],
        [0, 0, 2, 2],
    ]
)

box_2 = geotools.make_face(geotools.make_polygon(points, closed=True))
box_2 = geotools.extrude_shape(box_2, (0, 1, 0))

fused_boxes = geotools.boolean_fuse([box_1, box_2])

show_cad(fused_boxes)

cut_box_1 = geotools.boolean_cut(box_1, box_2)[0]

show_cad(cut_box_1)

## Modification of existing geometries

Now we're going to look at some stuff that we can do to change
geometries we've already made.
* Rotate
* Translate
* Scale

In [ ]:
# Let's save a deepcopy of a shape before modifying
new_cut_box_1 = cut_box_1.deepcopy()

new_cut_box_1.rotate(base=(0, 0, 0), direction=(0, 1, 0), degree=45)
new_cut_box_1.translate((0, 3, 0))
new_cut_box_1.scale(3)
blue_red_options = [DisplayCADOptions(color="blue"), DisplayCADOptions(color="red")]
show_cad([cut_box_1, new_cut_box_1], options=blue_red_options)

## Exporting geometry

At present, only the STEP Assembly format is supported
for exporting geometry.

In [ ]:
# Try saving any shape or group of shapes created above
# as a STEP assembly

my_shapes = [cut_box_1]
# Modify this file path to where you want to save the data.
my_file_path = "my_tutorial_assembly.STP"
geotools.save_as_STP(my_shapes, filename=my_file_path, scale=1)